In [1]:
!pip install --user --upgrade pip


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Requirement already up-to-date: pip in /home/jovyan/.local/lib/python3.6/site-packages (20.2.4)


In [2]:
!pip install --user kfp

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [3]:
import kfp
import kfp.dsl as dsl
import kfp.gcp as gcp
from kfp import components
import json
from string import Template
from jinja2 import Template as JinjaTemplate

In [4]:
def convert_result(result) -> str:
    import json
    hyperparameters = json.loads(result)
    args = []
    for param in hyperparameters:
        args.append(f"{param['name']}={param['value']}")
        return " ".join(args)

In [ ]:
KATIB_NAMESPACE = "kubeflow-olamideakinkunmi11"

 objectiveConfig = {
        "type": "maximize",
        "goal": goal,
        "objectiveMericName": "accuracy",
        "additionalMetricNames": ["rmse"],
    }
    
    algorithmConfig = {"algorithmName": "random"}
    
    metricCollectorSpec = {
        "collector": {
            "kind": "StdOut"
        }
    }
    
    parameters = [
        {"name": "--tf-learning-rate", "parameterType": "double", "feasibleSpace": {"min": "0.001", "max":"0.05"}},
    ]
    
    rawTemplate = {
        "apiVersion": "batch/v1",
        "kind": "Job",
        "metadata": {
            "name": "{{.Trial}}",
            "namespace": "{{.Namespace}}"
        },
        
        "spec": {
            "template": {
                "spec": {
                    "restartPolicy": "Never",
                    "containers": [
                        {
                            "name": "{{.Trial}}",
                            "image": str(training_image),
                            "imagePullPolicy": "Always",
                            "command": [
                                "python",
                                "train.py",
                                "--training-file-path={}".format(str(training_file_path)),
                                "--validation-file-path={}".format(str(validation_file_path)),
                                "{{- with .Hyperparameters}}",
                                "{{- range}}",
                                "{{.Name}}={{.Value}}",
                                "{{- end}}",
                                "{{- end}}"
                            ]
                        }
                    ]
                }
            }
        }
    }
    
    trialTemplate = {
        "goTemplate": {
            "rawTemplate": json.dumps(rawTemplate)
        }
    }

In [ ]:
def generate_sampling_query(source_table_name, num_lots, lots):
    sampling_query_template = """
        SELECT *
        FROM
            `{{ source_table }}` AS vehicle
        WHERE
        MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(vehicle)))), {{ num_lots }} IN ({{ lots }})
    """
    query = JinjaTemplate(sampling_query_template).render(
        source_table=source_table_name,
        num_lots=num_lots,
        lots=str(lots)[1:-1]
    )

    return query

In [ ]:
components_store = kfp.components.ComponentStore(
    local_search_path=None,
    url_search_prefix=[COMPONENT_URL_SEARCH_PREFIX]
)

bigquery_op = components_store.load_component('bigquery/query')
mlengine_train_op = components_store.load_component('ml_engine/train')
mlengine_deploy_op = components_store.load_component('ml_engine/deploy')

In [ ]:
@dsl.pipeline(
    name="Used-Cars",
    description="A pipeline to train and serve the used cars price prediction model"
)

def used_cars_pipeline(
    name="used-cars-{{workflow.id}}",
    katib_namespace=KATIB_NAMESPACE,
    goal=0.9,
    source_table_name=SOURCE_TABLE,
    # preprocess_image=f"gcr.io/{}/kubeflow-used-cars-preprocess:latest",
    training_image=f"gcr.io/{PROJECT_ID}/{TRAINING_IMAGE}",
    training_namespace="kubeflow",
    gcs_root="gs://used-cars/",
    parallel_trial_count=5,
    max_trial_count=3,
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    dataset_location="US"
    ):

    query = generate_sampling_query(
        source_table_name=source_table_name,
        num_lots=10,
        lots=[1, 2, 3, 4]
    )

    training_file_path = f'{gcs_root}/{TRAINING_FILE_PATH}'

    create_training_split = bigquery_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=training_file_path,
        dataset_location=dataset_location,
    )

    query = generate_sampling_query(
        source_table_name=source_table_name,
        num_lots=10,
        lots=[8]
    )

    validation_file_path = f'{gcs_root}/{VALIDATION_FILE_PATH}'

    create_validation_split = bigquery_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=validation_file_path,
        dataset_location=dataset_location,
    )
    query = generate_sampling_query(
        source_table_name=source_table_name,
        num_lots=10,
        lots=[9]
    )

    testing_file_path = f'{gcs_root}/{TESTING_FILE_PATH}'

    create_testing_split = bigquery_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=testing_file_path,
        dataset_location=dataset_location,
    )

    KATIB_NAMESPACE = "kubeflow-olamideakinkunmi11"
##################################KATIB SETTINGS #######################################################
    objectiveConfig = {
            "type": "maximize",
            "goal": goal,
            "objectiveMericName": "accuracy",
            "additionalMetricNames": ["rmse"],
        }
    
    algorithmConfig = {"algorithmName": "random"}
    
    metricCollectorSpec = {
        "collector": {
            "kind": "StdOut"
        }
    }
    
    parameters = [
        {"name": "--n-estimators", "parameterType": "integer", "feasibleSpace": {"min": "200", "max":"1000"}},
        {"name": "--min-samples-split", "parameterType": "integer", "feasibleSpace": {"min": "2", "max":"10"}},
        {"name": "--min-samples-leaf", "parameterType": "integer", "feasibleSpace": {"min": "1", "max":"4"}},
        {"name": "--max-features", "parameterType": "string", "feasibleSpace": ['auto', 'sqrt'],
        {"name": "--max-depth", "parameterType": "integer", "feasibleSpace": {"min": "10", "max":"100"}},
    ]
    
    rawTemplate = {
        "apiVersion": "batch/v1",
        "kind": "Job",
        "metadata": {
            "name": "{{.Trial}}",
            "namespace": "{{.Namespace}}"
        },
        
        "spec": {
            "template": {
                "spec": {
                    "restartPolicy": "Never",
                    "containers": [
                        {
                            "name": "{{.Trial}}",
                            "image": str(training_image),
                            "imagePullPolicy": "Always",
                            "command": [
                                "python",
                                "train.py",
                                "--training-file-path={}".format(create_training_split.outputs['output_gcs_path']),
                                "--validation-file-path={}".format(create_validation_split.outputs['output_gcs_path']),
                                "--hypertune=True",
                                "{{- with .Hyperparameters}}",
                                "{{- range}}",
                                "{{.Name}}={{.Value}}",
                                "{{- end}}",
                                "{{- end}}"
                            ]
                        }
                    ]
                }
            }
        }
    }
    
    trialTemplate = {
        "goTemplate": {
            "rawTemplate": json.dumps(rawTemplate)
        }
    }
#######################################################################################################


    
    
    katib_experiment_launcher_op = components.load_component_from_url(
        'https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/katib-launcher/component.yaml'
    )
    
    katib_op = katib_experiment_launcher_op(
        experiment_name=name,
        experiment_namespace=katib_namespace,
        parallel_trial_count=parallel_trial_count,
        max_trial_count=max_trial_count,
        objective=str(objectiveConfig),
        algorithm=str(algorithmConfig),
        trailTemplate=str(trialTemplate),
        parameters=str(parameters),
        metrics_collector=str(metricsCollectorSpec),
        delete_finished_experiment=False
    )
    
    
    convert_op = components.func_to_container_op(convert_result)
    op2 = convert_op(katib_op.output)

In [ ]:
pipeline = kfp.Client().create_run_from_pipeline_func(used_cars_pipeline, arguments={})